Copyright 2019 The TensorFlow Authors.

```
@title Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License.
You may obtain a copy of the License at
https://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
```

# Token hoá từ con

Hướng dẫn này trình bày làm thế nào để tạo ra một từ vựng từ con từ một tập dữ liệu, và sử dụng nó để xây dựng một `text.BertTokenizer` từ từ vựng.

Ưu điểm chính của tokenizer từ con là nó nội suy giữa token hóa dựa trên từ và trên ký tự. Các từ thông dụng có một vị trí trong từ vựng, nhưng tokenizer có thể rơi trở lại các mảnh từ và các ký tự riêng đối với các từ không xác định.

Mục tiêu: Vào cuối hướng dẫn này bạn sẽ xây dựng được hoàn toàn tokenizer từ đầu đến cuối mảnh từ và detokenizer từ tạp nham, và lưu nó như là một `saved_model` mà bạn có thể tải và sử dụng trong này [hướng dẫn dịch](https://tensorflow.org/text/tutorials/transformer).

## Tổng quan

Các gói `tensorflow_text` bao gồm triển khai TensorFlow của nhiều tokenizer chung. Điều này bao gồm ba loại tokenizers kiểu từ con:

* `text.BertTokenizer` - Lớp BertTokenizer là một giao diện cấp cao hơn. Nó bao gồm các thuật toán tách token của Bert và một `WordPieceTokenizer`. Nó lấy **các câu** như đầu vào và trả về các **token-ID**.

* `text.WordpieceTokenizer` - Lớp `WordPieceTokenizer` là một giao diện cấp thấp hơn. Nó chỉ thực hiện [thuật toán WordPiece](https://www.tensorflow.org/text/guide/subwords_tokenizer#applying_wordpiece). Bạn phải chuẩn hóa và tách văn bản thành các từ trước khi gọi nó. Nó lấy **các từ** như đầu vào và trả về các **token-ID**.

* `text.SentencepieceTokenizer` - `SentencepieceTokenizer` yêu cầu một thiết lập phức tạp hơn. Bộ khởi tạo của nó yêu cầu một mô hình mảnh câu tiền đào tạo. Xem [kho google/sentencepiece](https://github.com/google/sentencepiece#train-sentencepiece-model) để được hướng dẫn làm thế nào xây dựng một trong những mô hình này. Nó có thể chấp nhận **các câu** như đầu vào khi token hoá.

Hướng dẫn này xây dựng một từ vựng Mảnh từ theo cách từ trên xuống, bắt đầu từ các từ hiện có. Quá trình này không hoạt động ở tiếng Nhật, tiếng Trung hoặc tiếng Hàn vì những ngôn ngữ này không có các đơn vị nhiều ký tự rõ ràng. Để token hoá các ngôn ngữ đó cân nhắc sử dụng `text.SentencepieceTokenizer`, `text.UnicodeCharTokenizer` hoặc [cách tiếp cận này](https://tfhub.dev/google/zh_segmentation/1).

## Cài đặt



In [ ]:
!pip install -q -U tensorflow-text

In [ ]:
!pip install -q tensorflow_datasets

In [ ]:
import collections
import os
import pathlib
import re
import string
import sys
import tempfile
import time

import numpy as np
import matplotlib.pyplot as plt

import tensorflow_datasets as tfds
import tensorflow_text as text
import tensorflow as tf

In [ ]:
tf.get_logger().setLevel('ERROR')
pwd = pathlib.Path.cwd()

## Tải về tập dữ liệu

Lấy tập dữ liệu dịch tiếng Bồ Đào Nha/Anh từ [tfds](https://tensorflow.org/datasets):

In [ ]:
examples, metadata = tfds.load('ted_hrlr_translate/pt_to_en', with_info=True,
                               as_supervised=True)
train_examples, val_examples = examples['train'], examples['validation']

Tập dữ liệu này tạo ra các cặp câu tiếng Bồ Đào Nha/Anh:

In [ ]:
for pt, en in train_examples.take(1):
  print("Portuguese: ", pt.numpy().decode('utf-8'))
  print("English:   ", en.numpy().decode('utf-8'))

Lưu ý một số điều về các câu ví dụ ở trên:

* Chúng là chữ thường.
* Có khoảng trắng xung quanh dấu câu.
* Không rõ liệu chuẩn hóa unicode có đang được sử dụng hay không.

In [ ]:
train_en = train_examples.map(lambda pt, en: en)
train_pt = train_examples.map(lambda pt, en: pt)

## Tạo từ vựng

Phần này tạo ra một từ vựng mảnh từ từ một tập dữ liệu. Nếu bạn đã có một tập tin từ vựng và chỉ muốn xem làm thế nào để xây dựng một `text.BertTokenizer` hoặc tolenizer `text.Wordpiece` với nó thì bạn có thể bỏ qua thẳng đến phần [xây dựng các tokenizer](https://www.tensorflow.org/text/guide/subwords_tokenizer#build_the_tokenizer).

Lưu ý: Mã sinh từ vựng được sử dụng trong hướng dẫn này được tối ưu hóa cho **đơn giản**. Nếu bạn cần một giải pháp mở rộng hơn xem xét sử dụng triển khai Apache Beam sẵn có trong [tools/wordpiece_vocab/generate_vocab.py](https://github.com/tensorflow/text/blob/master/tensorflow_text/tools/wordpiece_vocab/generate_vocab.py)


Mã sinh từ vựng được bao gồm trong gói pip `tensorflow_text`. Nó không được nhập mặc định nên bạn cần phải nhập thủ công:

In [ ]:
from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset as bert_vocab

Hàm `bert_vocab.bert_vocab_from_dataset` sẽ tạo ra các từ vựng.

Có nhiều đối số bạn có thể đặt để điều chỉnh hành vi của nó. Đối với hướng dẫn này, bạn sẽ chủ yếu sử dụng các giá trị mặc định. Nếu bạn muốn tìm hiểu thêm về các tùy chọn, đầu tiên đọc [thuật toán](https://www.tensorflow.org/text/guide/subwords_tokenizer#optional_the_algorithm), và sau đó có nhìn vào [mã này](https://github.com/tensorflow/text/blob/master/tensorflow_text/tools/wordpiece_vocab/bert_vocab_from_dataset.py).

Quá trình này mất khoảng 2 phút.

In [ ]:
bert_tokenizer_params=dict(lower_case=True)
reserved_tokens=["[PAD]", "[UNK]", "[START]", "[END]"]

bert_vocab_args = dict(
    # The target vocabulary size
    vocab_size = 8000,
    # Reserved tokens that must be included in the vocabulary
    reserved_tokens=reserved_tokens,
    # Arguments for `text.BertTokenizer`
    bert_tokenizer_params=bert_tokenizer_params,
    # Arguments for `wordpiece_vocab.wordpiece_tokenizer_learner_lib.learn`
    learn_params={},
)

In [ ]:
%%time
pt_vocab = bert_vocab.bert_vocab_from_dataset(
    train_pt.batch(1000).prefetch(2),
    **bert_vocab_args
)

Đây là các vết cắt của kết quả từ vựng.

In [ ]:
print(pt_vocab[:10])
print(pt_vocab[100:110])
print(pt_vocab[1000:1010])
print(pt_vocab[-10:])

Viết tập tin từ vựng:

In [ ]:
def write_vocab_file(filepath, vocab):
  with open(filepath, 'w') as f:
    for token in vocab:
      print(token, file=f)

In [ ]:
write_vocab_file('pt_vocab.txt', pt_vocab)

Sử dụng hàm đó để tạo từ vựng từ dữ liệu tiếng Anh:

In [ ]:
%%time
en_vocab = bert_vocab.bert_vocab_from_dataset(
    train_en.batch(1000).prefetch(2),
    **bert_vocab_args
)

In [ ]:
print(en_vocab[:10])
print(en_vocab[100:110])
print(en_vocab[1000:1010])
print(en_vocab[-10:])

Đây là hai tập tin từ vựng:

In [ ]:
write_vocab_file('en_vocab.txt', en_vocab)

In [ ]:
!ls *.txt

## Xây dựng tokenizer

`text.BertTokenizer` có thể được khởi tạo bằng cách truyền đường dẫn của tập tin từ vựng làm đối số đầu tiên (xem phần [tf.lookup](https://www.tensorflow.org/text/guide/subwords_tokenizer#optional_tflookup) để biết các tùy chọn khác):

In [ ]:
pt_tokenizer = text.BertTokenizer('pt_vocab.txt', **bert_tokenizer_params)
en_tokenizer = text.BertTokenizer('en_vocab.txt', **bert_tokenizer_params)

Bây giờ bạn có thể sử dụng nó để mã hóa một số văn bản. Lấy một lô 3 ví dụ từ dữ liệu tiếng Anh:

In [ ]:
for pt_examples, en_examples in train_examples.batch(3).take(1):
  for ex in en_examples:
    print(ex.numpy())

Chạy nó thông qua phương thức `BertTokenizer.tokenize`. Ban đầu, nó trả về một `tf.RaggedTensor` với các trục `(batch, word, word-piece)`:

In [ ]:
# Tokenize the examples -> (batch, word, word-piece)
token_batch = en_tokenizer.tokenize(en_examples)
# Merge the word and word-piece axes -> (batch, tokens)
token_batch = token_batch.merge_dims(-2,-1)

for ex in token_batch.to_list():
  print(ex)

Nếu bạn thay thế các ID token bằng các biểu diễn văn bản của chúng (sử dụng `tf.gather`), bạn có thể thấy rằng trong ví dụ đầu tiên, các từ "`searchability`" và "`serendipity`" đã được phân tách thành "`search ##ability`" và "`s ##ere ##nd ##ip ##ity`":

In [ ]:
# Lookup each token id in the vocabulary.
txt_tokens = tf.gather(en_vocab, token_batch)
# Join with spaces.
tf.strings.reduce_join(txt_tokens, separator=' ', axis=-1)

Để tập hợp lại các từ từ các token được trích xuất, sử dụng phương thức `BertTokenizer.detokenize`:

> Lưu ý: `BertTokenizer.tokenize`/`BertTokenizer.detokenize` không khứ hồi losslessly. Kết quả của `detokenize` sẽ không, nói chung, có cùng một nội dung hoặc các bù đắp như đầu vào để `tokenize`. Điều này là do bước "token hoá cơ bản", để tách chuỗi thành các từ trước khi áp dụng `WordpieceTokenizer`, bao gồm các bước không thể đảo ngược như chữ thường và tách theo dấu câu. `WordpieceTokenizer` ở mặt khác **có thể đảo ngược**.


## Tùy chỉnh và xuất

Hướng dẫn này được xây dựng tokenizer văn bản và detokenizer sử dụng bởi các hướng dẫn [Transformer](https://tensorflow.org/text/tutorials/transformer). Phần này thêm các phương thức và các bước xử lý để đơn giản hóa hướng dẫn, và xuất các tokenizer sử dụng `tf.saved_model` để họ có thể được nhập bởi các hướng dẫn khác.

### Tuỳ chỉnh token hoá

Các hướng dẫn dưới đây mong đợi văn bản đã token hoá bao gồm các token `[START]` và `[END]`.

Các `reserved_tokens` giữ lại không gian ở phần đầu của từ vựng, vì vậy `[START]` và `[END]` có cùng các chỉ số ho cả hai ngôn ngữ:

In [ ]:
START = tf.argmax(tf.constant(reserved_tokens) == "[START]")
END = tf.argmax(tf.constant(reserved_tokens) == "[END]")

def add_start_end(ragged):
  count = ragged.bounding_shape()[0]
  starts = tf.fill([count,1], START)
  ends = tf.fill([count,1], END)
  return tf.concat([starts, ragged, ends], axis=1)

In [ ]:
words = en_tokenizer.detokenize(add_start_end(token_batch))
tf.strings.reduce_join(words, separator=' ', axis=-1)

### Tuỳ chỉnh detoken hoá

Trước khi xuất các `tokenizer`, có một số thứ bạn có thể dọn dẹp cho các hướng dẫn phía dưới:

1. Họ muốn tạo đầu ra văn bản sạch, vì vậy thả tokens để dành như `[START]`, `[END]` và `[PAD]`.

2. Họ đang quan tâm trong các chuỗi hoàn chỉnh, vì vậy áp dụng một chuỗi nối dọc theo trục `words` của kết quả.

In [ ]:
def cleanup_text(reserved_tokens, token_txt):
  # Drop the reserved tokens, except for "[UNK]".
  bad_tokens = [re.escape(tok) for tok in reserved_tokens if tok != "[UNK]"]
  bad_token_re = "|".join(bad_tokens)
    
  bad_cells = tf.strings.regex_full_match(token_txt, bad_token_re)
  result = tf.ragged.boolean_mask(token_txt, ~bad_cells)

  # Join them into strings.
  result = tf.strings.reduce_join(result, separator=' ', axis=-1)

  return result

In [ ]:
en_examples.numpy()

In [ ]:
token_batch = en_tokenizer.tokenize(en_examples).merge_dims(-2,-1)
words = en_tokenizer.detokenize(token_batch)
words

In [ ]:
cleanup_text(reserved_tokens, words).numpy()

### Xuất

Khối mã sau xây dựng một lớp `CustomTokenizer` để chứa các thể hiện `text.BertTokenizer`, logic tùy chỉnh, và lớp bọc `@tf.function` cần thiết cho xuất.

In [ ]:
class CustomTokenizer(tf.Module):
  def __init__(self, reserved_tokens, vocab_path):
    self.tokenizer = text.BertTokenizer(vocab_path, lower_case=True)
    self._reserved_tokens = reserved_tokens
    self._vocab_path = tf.saved_model.Asset(vocab_path)

    vocab = pathlib.Path(vocab_path).read_text().splitlines()
    self.vocab = tf.Variable(vocab)

    ## Create the signatures for export:   

    # Include a tokenize signature for a batch of strings. 
    self.tokenize.get_concrete_function(
        tf.TensorSpec(shape=[None], dtype=tf.string))
    
    # Include `detokenize` and `lookup` signatures for:
    #   * `Tensors` with shapes [tokens] and [batch, tokens]
    #   * `RaggedTensors` with shape [batch, tokens]
    self.detokenize.get_concrete_function(
        tf.TensorSpec(shape=[None, None], dtype=tf.int64))
    self.detokenize.get_concrete_function(
          tf.RaggedTensorSpec(shape=[None, None], dtype=tf.int64))

    self.lookup.get_concrete_function(
        tf.TensorSpec(shape=[None, None], dtype=tf.int64))
    self.lookup.get_concrete_function(
          tf.RaggedTensorSpec(shape=[None, None], dtype=tf.int64))

    # These `get_*` methods take no arguments
    self.get_vocab_size.get_concrete_function()
    self.get_vocab_path.get_concrete_function()
    self.get_reserved_tokens.get_concrete_function()
    
  @tf.function
  def tokenize(self, strings):
    enc = self.tokenizer.tokenize(strings)
    # Merge the `word` and `word-piece` axes.
    enc = enc.merge_dims(-2,-1)
    enc = add_start_end(enc)
    return enc

  @tf.function
  def detokenize(self, tokenized):
    words = self.tokenizer.detokenize(tokenized)
    return cleanup_text(self._reserved_tokens, words)

  @tf.function
  def lookup(self, token_ids):
    return tf.gather(self.vocab, token_ids)

  @tf.function
  def get_vocab_size(self):
    return tf.shape(self.vocab)[0]

  @tf.function
  def get_vocab_path(self):
    return self._vocab_path

  @tf.function
  def get_reserved_tokens(self):
    return tf.constant(self._reserved_tokens)

Xây dựng một `CustomTokenizer` cho mỗi ngôn ngữ:

In [ ]:
tokenizers = tf.Module()
tokenizers.pt = CustomTokenizer(reserved_tokens, 'pt_vocab.txt')
tokenizers.en = CustomTokenizer(reserved_tokens, 'en_vocab.txt')

Xuất các tokenizer dưới dạng `save_model`:

In [ ]:
model_name = 'ted_hrlr_translate_pt_en_converter'
tf.saved_model.save(tokenizers, model_name)

Tải lại `saved_model` và kiểm tra các phương thức:

In [ ]:
reloaded_tokenizers = tf.saved_model.load(model_name)
reloaded_tokenizers.en.get_vocab_size().numpy()

In [ ]:
tokens = reloaded_tokenizers.en.tokenize(['Hello TensorFlow!'])
tokens.numpy()

In [ ]:
text_tokens = reloaded_tokenizers.en.lookup(tokens)
text_tokens

In [ ]:
round_trip = reloaded_tokenizers.en.detokenize(tokens)

print(round_trip.numpy()[0].decode('utf-8'))

Lưu trữ nó cho [các hướng dẫn dịch](https://tensorflow.org/text/tutorials/transformer):

In [ ]:
!zip -r {model_name}.zip {model_name}

In [ ]:
!du -h *.zip

## Tuỳ chọn: Thuật toán

Điều đáng chú ý ở đây là có hai phiên bản của thuật toán WordPiece: Dưới-lên và trên-xuống. Trong cả hai trường hợp, mục tiêu là như nhau: "*Với một kho ngữ liệu đào tạo và một số token D mong muốn, vấn đề tối ưu hóa là chọn các mảnh từ D sao cho kho ngữ liệu thu được là tối thiểu về số lượng mảnh từ chữ khi được phân đoạn theo mô hình mảnh từ đã chọn.*"

Bản gốc [thuật toán WordPiece từ dưới-lên](https://static.googleusercontent.com/media/research.google.com/ja//pubs/archive/37842.pdf), dựa trên [mã hóa cặp byte](https://towardsdatascience.com/byte-pair-encoding-the-dark-horse-of-modern-nlp-eb36c7df4f10) . Giống như BPE, Nó bắt đầu với bảng chữ cái và kết hợp lặp đi lặp lại các bigram thông thường để tạo thành các mảnh từ và từ.

Bộ tạo từ vựng của TensorFlow Text theo sau việc thực hiện từ trên-xuống từ [Bert](https://arxiv.org/pdf/1810.04805.pdf). Bắt đầu với các từ và chia nhỏ chúng thành các thành phần nhỏ hơn cho đến khi chúng đạt đến ngưỡng tần suất, hoặc không thể chia nhỏ hơn nữa. Phần tiếp theo mô tả chi tiết điều này. Đối với tiếng Nhật, tiếng Trung và tiếng Hàn, cách tiếp cận từ trên-xuống này không hoạt động vì không có đơn vị từ rõ ràng nào để bắt đầu. Đối với những bạn cần một [cách tiếp cận khác](https://tfhub.dev/google/zh_segmentation/1).

(Phần sau dài quá nên lười... 😁)

### Choosing the vocabulary

The top-down WordPiece generation algorithm takes in a set of (word, count) pairs and a threshold `T`, and returns a vocabulary `V`.

The algorithm is iterative. It is run for `k` iterations, where typically `k = 4`, but only the first two are really important. The third and fourth (and beyond) are just identical to the second. Note that each step of the binary search runs the algorithm from scratch for `k` iterations.

The iterations described below:

#### First iteration

1.  Iterate over every word and count pair in the input, denoted as `(w, c)`.
2.  For each word `w`, generate every substring, denoted as `s`. E.g., for the
    word `human`, we generate `{h, hu, hum, huma,
    human, ##u, ##um, ##uma, ##uman, ##m, ##ma, ##man, #a, ##an, ##n}`.
3.  Maintain a substring-to-count hash map, and increment the count of each `s`
    by `c`. E.g., if we have `(human, 113)` and `(humas, 3)` in our input, the
    count of `s = huma` will be `113+3=116`.
4.  Once we've collected the counts of every substring, iterate over the `(s,
    c)` pairs *starting with the longest `s` first*.
5.  Keep any `s` that has a `c > T`. E.g., if `T = 100` and we have `(pers,
    231); (dogs, 259); (##rint; 76)`, then we would keep `pers` and `dogs`.
6.  When an `s` is kept, subtract off its count from all of its prefixes. This
    is the reason for sorting all of the `s` by length in step 4. This is a
    critical part of the algorithm, because otherwise words would be double
    counted. For example, let's say that we've kept `human` and we get to
    `(huma, 116)`. We know that `113` of those `116` came from `human`, and `3`
    came from `humas`. However, now that `human` is in our vocabulary, we know
    we will never segment `human` into `huma ##n`. So once `human` has been
    kept, then `huma` only has an *effective* count of `3`.

This algorithm will generate a set of word pieces `s` (many of which will be
whole words `w`), which we *could* use as our WordPiece vocabulary.

However, there is a problem: This algorithm will severely overgenerate word
pieces. The reason is that we only subtract off counts of prefix tokens.
Therefore, if we keep the word `human`, we will subtract off the count for `h,
hu, hu, huma`, but not for `##u, ##um, ##uma, ##uman` and so on. So we might
generate both `human` and `##uman` as word pieces, even though `##uman` will
never be applied.

So why not subtract off the counts for every *substring*, not just every
*prefix*? Because then we could end up subtracting off the counts multiple
times. Let's say that we're processing `s` of length 5 and we keep both
`(##denia, 129)` and `(##eniab, 137)`, where `65` of those counts came from the
word `undeniable`. If we subtract off from *every* substring, we would subtract
`65` from the substring `##enia` twice, even though we should only subtract
once. However, if we only subtract off from prefixes, it will correctly only be
subtracted once.

### Applying WordPiece

<a id="applying_wordpiece"></a>

Once a WordPiece vocabulary has been generated, we need to be able to apply it
to new data. The algorithm is a simple greedy longest-match-first application.

For example, consider segmenting the word `undeniable`.

We first lookup `undeniable` in our WordPiece dictionary, and if it's present,
we're done. If not, we decrement the end point by one character, and repeat,
e.g., `undeniabl`.

Eventually, we will either find a subtoken in our vocabulary, or get down to a
single character subtoken. (In general, we assume that every character is in our
vocabulary, although this might not be the case for rare Unicode characters. If
we encounter a rare Unicode character that's not in the vocabulary we simply map
the entire word to `<unk>`).

In this case, we find `un` in our vocabulary. So that's our first word piece.
Then we jump to the end of `un` and repeat the processing, e.g., try to find
`##deniable`, then `##deniabl`, etc. This is repeated until we've segmented the
entire word.

## Optional: tf.lookup

<a id="tf.lookup"></a>

If you need access to, or more control over the vocabulary it's worth noting that you can build the lookup table yourself and pass that to `BertTokenizer`.

When you pass a string, `BertTokenizer` does the following:

In [ ]:
pt_lookup = tf.lookup.StaticVocabularyTable(
    num_oov_buckets=1,
    initializer=tf.lookup.TextFileInitializer(
        filename='pt_vocab.txt',
        key_dtype=tf.string,
        key_index = tf.lookup.TextFileIndex.WHOLE_LINE,
        value_dtype = tf.int64,
        value_index=tf.lookup.TextFileIndex.LINE_NUMBER)) 
pt_tokenizer = text.BertTokenizer(pt_lookup)

Now you have direct access to the lookup table used in the tokenizer.

In [ ]:
pt_lookup.lookup(tf.constant(['é', 'um', 'uma', 'para', 'não']))

You don't need to use a vocabulary file, `tf.lookup` has other initializer options. If you have the vocabulary in memory you can use `lookup.KeyValueTensorInitializer`:

In [ ]:
pt_lookup = tf.lookup.StaticVocabularyTable(
    num_oov_buckets=1,
    initializer=tf.lookup.KeyValueTensorInitializer(
        keys=pt_vocab,
        values=tf.range(len(pt_vocab), dtype=tf.int64))) 
pt_tokenizer = text.BertTokenizer(pt_lookup)